<a href="https://colab.research.google.com/github/Rogerio-mack/IMT_CD_2025/blob/main/IA_HuggingFace_e_Agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Llama, HuggingFace, LangChain**

In [ ]:
# para rodar no Colab
!pip install transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

#### You also need set up GPU in Colab environment

# https://huggingface.co/meta-llama/Llama-2-7b-chat-hf

<!-- token: hf_QftvNgknCQ.lyEIZ.zqkVkTjSRCIgLvfQE aA -->

In [ ]:
from huggingface_hub import login

In [ ]:
login()

# **(a) Resposta de prompt com modelo LLaMA (HuggingFace)**

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Use o modelo LLaMA 2 da Meta (necessário login no HuggingFace)
model_id = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
#    load_in_8bit=True  # para rodar no Colab com GPU
)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

prompt = "Explique o que é inteligência artificial em termos simples."
resposta = pipe(prompt, max_new_tokens=100, do_sample=True, temperature=0.7)
print(resposta[0]['generated_text'])


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Device set to use cpu


KeyboardInterrupt: 

# **(b) Conversa encadeada com LLaMA + LangChain**

In [ ]:
!pip install langchain

In [ ]:
from langchain.chat_models import HuggingFaceChatModel
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

# Configure o modelo de linguagem da Hugging Face
llama_model = HuggingFaceChatModel(
    model_name="meta-llama/Llama-2-7b-chat-hf",
    model_kwargs={
        "device": "cuda"  # ou "cpu" se não tiver GPU
    }
)

# Crie uma memória para manter o contexto do diálogo
memory = ConversationBufferMemory()

# Crie uma cadeia de conversa
conversation = ConversationChain(
    llm=llama_model,
    memory=memory,
)


# **(c) Fine-tuning com TorchTune com novos textos**

![imagem](https://github.com/Rogerio-mack/IA_2025S1/blob/main/figures/LoRA_QLoRA.jpg?raw=true)

| Característica        | LoRA                           | QLoRA                                      |
|-----------------------|--------------------------------|--------------------------------------------|
| Técnicas combinadas  | Baixa-rank + fine-tuning       | Baixa-rank + quantização (4 bits ou menos) |
| Uso de memória      | Reduzido comparado ao fine-tuning completo | Ainda mais reduzido por quantização       |
| Eficiência          | Alta                           | Muito alta, possibilitando treinar modelos maiores em hardware restrito |
| Complexidade        | Menos complexa                | Mais complexa, devido à quantização       |


Atenção, esse código não deve executar no Colab pois o TorchTune está em desenvolvimento e precisa executar local.

In [ ]:
YAML = '''
model:
  _component_: torchtune.models.llama2.llama2_7b
  pretrained: true
  path: /meta-llama/Llama-2-7b-chat-hf

tokenizer:
  _component_: torchtune.tokenizers.llama2_tokenizer
  pretrained_model_name_or_path: meta-llama/Llama-2-7b-hf

dataset:
  _component_: torchtune.datasets.instruct_dataset
  name: train
  format: folder
  source: dataset.jsonl
  template: torchtune.data.AlpacaInstructTemplate
  max_seq_len: 512
  train_on_input: true

trainer:
  _component_: torchtune.trainers.SFTTrainer
  max_steps: 100
  eval_steps: 10
  save_steps: 50


'''


In [ ]:
with open("finetune_config.yaml", "w") as f:
  f.write(YAML)

!cat finetune_config.yaml


model:
  _component_: torchtune.models.llama2.llama2_7b
  pretrained: true
  path: /meta-llama/Llama-2-7b-chat-hf

tokenizer:
  _component_: torchtune.tokenizers.llama2_tokenizer
  pretrained_model_name_or_path: meta-llama/Llama-2-7b-hf

dataset:
  _component_: torchtune.datasets.instruct_dataset
  source: dataset.jsonl
  template: torchtune.data.AlpacaInstructTemplate
  max_seq_len: 512
  train_on_input: true

trainer:
  _component_: torchtune.trainers.SFTTrainer
  max_steps: 100
  eval_steps: 10
  save_steps: 50




In [ ]:
import torch
print(torch.cuda.is_available())


True


In [ ]:
# !pip install --upgrade torchtune


In [ ]:
# !pip install torchtune transformers datasets accelerate bitsandbytes

In [ ]:
# !tune run full_finetune_single_device --config finetune_config.yaml

# **Agentes de IA**

São sistemas de IA mais complexos que os LLMs e que podem perceber o ambiente, tomar decisões e agir com base em objetivos específicos. Em geral, eles têm capacidade de aprender e melhorar ao longo do tempo com base nas nossas interações. Geralmente, eles combinam LLMs com componentes de percepção, raciocínio, planejamento, e controle de ações no ambiente, podendo interagir com usuários, objetos ou sistemas externos.

<br>

| Característica	| LLMs	| Agentes de IA |
|-----------------|-------|---------------|
| Função Principal	| Geração e compreensão de texto	| Execução autônoma de tarefas |
| Exemplo	| GPT-3, LLaMA	| Google Assistant, Chatbots de suporte |
| Treinamento	| Treinados com grandes datasets de texto	|Podem ser treinados ou refinados através de interação contínua |
|Interação com o Usuário	|Fornecem respostas baseadas em prompts |	Interagem ativamente com usuários para resolver questões sendo proativos |
| Capacidade de Aprendizado	| Aprendem padrões linguísticos |	Podem melhorar com experiência e feedback |
| Uso de Dados|	Baseiam-se em dados textuais	|Usam dados estruturados e não estruturados, podendo incluir dados sensoriais |


<br>

Os agentes de IA incluem sistemas de recomendação, sistemas de veículos autônomos, e mais recentemente existem várias ferramentas e frameworks que funcionam como agentes de IA empregando LLMs. O **LangChain** e **AutoGPT** são frameworks que permitem criar e personalizar agentes de IA com modelos de linguagem avançados, e **GenSpark** e **Manus AI**, são focados em auxiliar na automação de tarefas e  pesquisa web e documentos.  






# **Regularização**

Objetivo do treinamento é minimizar as perdas, ou seja:

$$\text{minimizar(perda)}$$

modelos focados exclusivamente em minimizar perdas tendem a apresentar overfitting. Um algoritmo de otimização de treinamento melhor minimiza alguma combinação de perdas e complexidade:

$$\text{minimizar(perda + complexidade)}$$

É o que chamamos de regularização.
